In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
code = """
#include <stdio.h>
#include <stdlib.h>
#define N 1000000
__global__ void add(int *a, int *b, int *c) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < N) {
c[tid] = a[tid] + b[tid];
}
}
int main() {
int *a, *b, *c;
int *d_a, *d_b, *d_c;
int size = N * sizeof(int);
// Allocate memory on host
a = (int*)malloc(size);
b = (int*)malloc(size);
c = (int*)malloc(size);
// Initialize arrays
for (int i = 0; i < N; i++) {
a[i] = i;
b[i] = i * 2;
}
// Allocate memory on device
cudaMalloc(&d_a, size);
cudaMalloc(&d_b, size);
cudaMalloc(&d_c, size);
// Copy data from host to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
// Launch kernel with 1 million threads
add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);
// Copy result from device to host
cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
// Print first and last elements of result
printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);
// Free memory
free(a);
free(b);
free(c);
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}
"""

In [ ]:
text_file = open("assign4.cu","w")
text_file.write(code)
text_file.close()

In [ ]:
!nvcc assign4.cu

In [ ]:
!./a.out

c[0]=0, c[999999] = 2999997

In [ ]:
!nvprof ./a.out

==1799== NVPROF is profiling process 1799, command: ./a.out
c[0]=0, c[999999] = 2999997==1799== Profiling application: ./a.out
==1799== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   52.00%  1.7214ms         1  1.7214ms  1.7214ms  1.7214ms  [CUDA memcpy DtoH]
                   46.57%  1.5416ms         2  770.81us  739.51us  802.10us  [CUDA memcpy HtoD]
                    1.43%  47.487us         1  47.487us  47.487us  47.487us  add(int*, int*, int*)
      API calls:   97.26%  251.03ms         3  83.676ms  88.460us  250.84ms  cudaMalloc
                    1.97%  5.0869ms         3  1.6956ms  971.97us  3.1279ms  cudaMemcpy
                    0.47%  1.2011ms         1  1.2011ms  1.2011ms  1.2011ms  cuDeviceGetPCIBusId
                    0.22%  561.46us         3  187.15us  144.21us  209.07us  cudaFree
                    0.06%  160.24us       101  1.5860us     257ns  64.117us  cuDeviceGetAttribute
                